In [ ]:
#script to web scrape the hansard website.
#1-filter by document type speeches
#2-get the list of all MPs from the 47 parliament onwards
#3-download all speeches from each MP, once the MP name is in the title of the document

In [ ]:
import bs4 #parse html from web pages
from lxml import html
import requests #load web pages
from urllib2 import urlopen #load web pages
import re
import mechanize
import time
import os
import csv

In [ ]:
#using request as loader and BeautifulSoup as parser
#http://blog.miguelgrinberg.com/post/easy-web-scraping-with-python
#4721 pages remaining (total of 94429 documents)

root_url = 'https://www.parliament.nz'
index_url = root_url + '/en/pb/hansard-debates/rhr/search'
search_url = index_url + '?Criteria.PageNumber='

In [ ]:
#function to write list of url to txt file
def list_txt_file(filename,z):
    f = open('/home/dvasques/Dropbox (Complex systems)/TPM Research Bootcamp/speech-data/' + filename + '.txt', 'wb')
    for url in z:
        url = url.encode('utf-8').strip()
        f.write("%s\n" % url)
    f.close()

In [ ]:
#function to read url from file and create a list
def url_from_csv(filename):
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        speech_urls = list(reader)
    
    return speech_urls

In [ ]:
#function to get all the MPs from Hansard's search website
def get_mps_list(index_url):
    
    t0 = time.time()
    
    response = requests.get(index_url)
    soup = bs4.BeautifulSoup(response.content,'lxml')
    mps_list = []
    for option in soup.select('select#criteria_MemberOfParliament option[value]'):
        mps_list.append(option.attrs.get('value'))
        
    print 'Running time to get MPs list: ' + str(time.time() - t0)
        
    return mps_list

In [ ]:
#function to create folders with MPs' names
def create_mp_folder(mps_list):
    root_path = '/home/dvasques/Desktop/speech-data-d/'
    folder = mps_list
    os.mkdir(os.path.join(root_path,folder))
    #for folder in folders:
        #os.mkdir(os.path.join(root_path,folder))

In [ ]:
#function to write the extracted text to a txt tile
def write_txt_file(filename,z):
    f = open('/home/dvasques/Desktop/speech-data-d/' + filename + '.txt', 'wb')
    f.write(z)
    f.close()

In [ ]:
#function to find positioning of dash (to get only the mp name)
def dash_positions(url):
    
    s = url
    c = '-'
    positions = [pos for pos, char in enumerate(s) if char == c]
    #print speech_urls[205][0].index('-')
    
    return positions

In [ ]:
#function to write all url content into txt files
def speech_to_txt(speech_urls):
    
    t0 = time.time()
    
    for i in range(len(speech_urls)):
    #for i in range(0,100,1):
        
        response = requests.get(speech_urls[i][0])
        soup = bs4.BeautifulSoup((response.content),'lxml')
        z = s.find("div", {"class": "section"}).get_text()
        zencode = z.encode('utf-8').strip()
        #for strong_tag in z[0].find_all('strong'):
            #print strong_tag.text
        filename = speech_urls[i][61:86] + '-' + speech_urls[i][87:]
        write_txt_file(filename,zencode)
        #print strong_tag
        
        print i
        
    print 'Running time to get all documents: ' + str(time.time() - t0)

In [ ]:
#function to pre-process raw data, writing content organized in folders
def speech_to_txt2(speech_urls, parl, year, date, key, mps, mps_dict): #with lists
    #with lists is easier to create nodes with attributes
    
    t0 = time.time()
    
    #for i in range(len(speech_urls)):
    for i in range(0,len(speech_urls),1):
        
        url = speech_urls[i][0]
    
        #when mp name starts with digit it is not a valid speech from MP
        if len(url) > 120:
        
            if not speech_urls[i][0][120].isdigit():

                #get speech if url has text
                if i < 494:
                    original_url = speech_urls[i][0][:94] + speech_urls[i][0][96:118] + '0' + speech_urls[i][0][118:]
                    response = requests.get(original_url)
                else:     
                    response = requests.get(url)

                soup = bs4.BeautifulSoup((response.content),'lxml')
                z = soup.find("div", {"class": "section"})
                if z is not None:
                    z = soup.find("div", {"class": "section"}).get_text()
                    zencode = z.encode('utf-8').strip()



                    parl_s = url[94:96]
                    parl.append(parl_s)
                    year_s = url[102:106]
                    year.append(year_s)
                    date_s = url[102:110]
                    date.append(date_s)
                    key_s = url[111:119]
                    key.append(key_s)


                    #find positions of each dash in the url, calling from function
                    positions = dash_positions(url)
                    if len(positions)>3:
                        mp_name = url[120:positions[3]] #string
                        mps.append(mp_name)
                    else:
                        mp_name = url[120:]
                        mps.append(mp_name)


                    #creating a dictionary of urls for each mp
                    if mp_name not in mps_dict.keys():
                        create_mp_folder(mp_name)
                        mps_dict[mp_name] = [url]
                    else:
                        mps_dict[mp_name].append(url) 


                    filename = mp_name + '/' + parl_s + '-' + date_s + '-' + key_s
                    write_txt_file(filename,zencode)
                    #print strong_tag

                    print i
                
            else:
                print 'No text: ', speech_urls[i][0]
        
        else:
            print speech_urls[i][0]
        
    print 'Running time to get all documents: ' + str(time.time() - t0)
    
    return parl, year, date, key, mps, mps_dict

In [ ]:
#get urls from csv file
speech_urls = url_from_csv('/home/dvasques/Desktop/speech_urls.csv')
print 'Number of urls: ', len(speech_urls)

In [ ]:
#correct urls to extract information from url strings
for i in range(0,494,1):
    speech_urls[i][0] = speech_urls[i][0][:94] + '51' + speech_urls[i][0][94:117] + speech_urls[i][0][118:]

print speech_urls[0][0]
print speech_urls[0][0][:94] + speech_urls[0][0][96:118] + '0' + speech_urls[0][0][118:]

print speech_urls[493][0]
print speech_urls[494][0]

In [ ]:
#initialize lists and dictionary
parl = []
year = []
date = []
key = []
mps = []
mps_dict = {}

In [ ]:
#call function
parl, year, date, key, mps, mps_dict = speech_to_txt2(speech_urls, parl, year, date, key, mps, mps_dict)